In [18]:
import requests
import pandas as pd
import time
import calendar  # Import the calendar module

# Base URL and API key
base_url = "https://api.weatherbit.io/v2.0/history/hourly"
api_key = "75160ed1474f4a8896de726521bdcefe"

# Define the latitude and longitude
lat = "47.65"
lon = "-122.30"

# Months and years to query
months = range(1, 13)  # All months (1-12)
years = range(2024, 2025)  # From 2022 to 2023 for the example

# Loop over each year and month
for year in years:
    dataframes = []  # List to collect each month's DataFrame for the current year
    for month in months:
        # Use the calendar module to find the last day of the month
        last_day = calendar.monthrange(year, month)[1]
        start_date = f"{year}-{month:02d}-01"
        end_date = f"{year}-{month:02d}-{last_day}"
        
        # Construct URL with parameters
        url = f"{base_url}?lat={lat}&lon={lon}&start_date={start_date}%3A00&end_date={end_date}%3A00&tz=local&key={api_key}"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            if 'data' in data:
                df = pd.DataFrame(data['data'])
                dataframes.append(df)
                print(f"Data collected for {start_date} to {end_date}")
        else:
            print(f"Failed to retrieve data for {start_date} to {end_date}: {response.status_code}")

        # Delay between requests
        time.sleep(5)  # Sleep for 5 seconds

    # Concatenate all monthly dataframes for the current year
    if dataframes:
        combined_df = pd.concat(dataframes, ignore_index=True)
        combined_df.to_csv(f'weather_data_{year}.csv', index=False)
        print(f"Data for the year {year} saved successfully.")

# Now merge all yearly data files into a single DataFrame
all_years_data = []
for year in years:
    df = pd.read_csv(f'weather_data_{year}.csv')
    all_years_data.append(df)

final_df = pd.concat(all_years_data, ignore_index=True)
final_df.to_csv('weather_data_all_years.csv', index=False)
print("All years merged into one file 'weather_data_all_years.csv'.")




Failed to retrieve data for 2024-01-01 to 2024-01-31: 429
Failed to retrieve data for 2024-02-01 to 2024-02-29: 429


KeyboardInterrupt: 

In [17]:
import pandas as pd

file_path = 'QUAD_RAW.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

def clean_and_convert(df):
    # Date and time transformation
    df['date'] = pd.to_datetime(df['timestamp_local']).dt.date
    df['time'] = pd.to_datetime(df['timestamp_local']).dt.time

    # Rename columns
    df.rename(columns={
        'clouds': 'Cloud Coverage (%)',
        'temp': 'Temperature (Celcius)',
        'wind_spd': 'Wind speed (m/s)',
        'precip': 'rain (mm)',
        'rh': 'relative humidity (%)',
        'vis': 'visibility (km)'
    }, inplace=True)

    # Convert units
    df['Temperature (Fahrenheit)'] = df['Temperature (Celcius)'] * 9/5 + 32
    df['Wind speed (mph)'] = df['Wind speed (m/s)'] * 2.237
    df['rain (inches)'] = df['rain (mm)'] * 0.0393701
    df['visibility (miles)'] = df['visibility (km)'] * 0.621371

    df.fillna(0, inplace=True)

    # Select and keep only the necessary columns
    df = df[['date', 'time', 'Cloud Coverage (%)', 'Temperature (Fahrenheit)', 'Wind speed (mph)',
             'rain (inches)', 'relative humidity (%)', 'visibility (miles)']]
    
    numerical_cols = ['Cloud Coverage (%)', 'Temperature (Fahrenheit)', 'Wind speed (mph)',
                      'rain (inches)', 'relative humidity (%)', 'visibility (miles)']
    df[numerical_cols] = df[numerical_cols].round(2)

    return df


# Clean and convert the combined data
cleaned_df = clean_and_convert(df)

# Export to CSV
cleaned_df.to_csv('weather_data_2013_to_2023_cleaned.csv', index=False)

print("Data collected, cleaned, and saved to 'weather_data_2013_to_2023_cleaned.csv'.")


Data collected, cleaned, and saved to 'weather_data_2013_to_2023_cleaned.csv'.


/var/folders/cq/syt8sl2j3hd_7zb95ds5xmmh0000gn/T/ipykernel_14546/2396404256.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numerical_cols] = df[numerical_cols].round(2)
